In [ ]:
import optuna

In [ ]:
def create_model(trial):
    # Suggest hyperparameters
    filters_layer_1 = trial.suggest_int('filters_layer_1', 16, 64, step=16)
    filters_layer_2 = trial.suggest_int('filters_layer_2', 32, 128, step=32)
    units_dense_1 = trial.suggest_int('units_dense_1', 64, 256, step=64)
    units_dense_2 = trial.suggest_int('units_dense_2', 16, 128, step=16)
    dropout_rate = trial.suggest_float('dropout_rate', 0.3, 0.7)
    optimizer = trial.suggest_categorical('optimizer', ['adam', 'sgd', 'rmsprop'])
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])

    # Build model
    model = keras.Sequential([
        layers.Conv2D(filters_layer_1, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(filters_layer_2, kernel_size=(3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(units_dense_1, activation='relu'),
        layers.Dropout(dropout_rate),
        layers.Dense(units_dense_2, activation='softmax')
    ])

    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    history = model.fit(X, y, validation_split=0.2, batch_size=batch_size, epochs=5, verbose=10)

    # Return the best validation accuracy
    val_accuracy = max(history.history['val_accuracy'])
    return val_accuracy

In [ ]:
# Create a study
study = optuna.create_study(direction='maximize')  # We want to maximize validation accuracy
study.optimize(create_model, n_trials=50)  # Perform 50 trials

# Print the best hyperparameters and score
print("Best Hyperparameters: ", study.best_params)
print("Best Validation Accuracy: ", study.best_value)